In [121]:
import json
import os
import numpy as np
from pprint import pprint
from copy import deepcopy
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, auc

import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
from torchvision.ops import box_iou
from torchvision.ops import nms

import matplotlib.pyplot as pyplot
from ensemble_boxes import non_maximum_weighted

from scipy.special import softmax
from scipy.special import expit

In [122]:
def compute_ap(recall, precision):
    """ Compute the average precision, given the recall and precision curves.
    Code originally from https://github.com/rbgirshick/py-faster-rcnn.
    # Arguments
        recall:    The recall curve (list).
        precision: The precision curve (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.], recall, [1.]))
    mpre = np.concatenate(([0.], precision, [0.]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

In [123]:
anno_list = [ 557,  600,  614,  547,  503]
case_list = [1193, 1185, 1192, 1175, 1147]

In [129]:
model_path = '../val_output/N4_All/'
pprint([each for each in sorted(os.listdir(model_path))])

['MCS_NL23_bs16_iou03_size448',
 'MCS_NL3_bs16_iou03_size448',
 'MCSdv0123_bs16_iou03_size448',
 'MCSdv123_bs16_iou03_size448',
 'MCSdv23_bs16_iou03_size448',
 'MCSdv23_bs16_iou03_size896',
 'MCSdv3_bs16_iou03_size448',
 'MC_CO_dv0123_bs16_iou03_size448',
 'MC_CO_dv123_bs16_iou03_size448',
 'MC_CO_dv123_bs16_iou03_size896',
 'MC_CO_dv23_bs16_iou03_size448',
 'MC_CO_dv23_bs16_iou03_size896',
 'MC_CO_dv3_bs16_iou03_size448',
 'MC_CO_dv3_bs16_iou03_size896',
 'MC_SE_dv23_bs16_iou03_size896',
 'MC_SE_dv3_bs16_iou03_size896',
 'MCpre_bs16_iou03_size672',
 'MCpre_bs16_iou03_size896',
 'MP_b012_CO_dv0123_bs16_iou03_size896',
 'mask_rcnn_1',
 'mask_rcnn_FLAIR',
 'mask_rcnn_T1',
 'mask_rcnn_T1T2',
 'mask_rcnn_T2',
 'mask_rcnn_T2FLAIR',
 'mrDS_bs16_iou03_size892',
 'mrSAdv23_bs16_iou03_size448',
 'mrSAdv23_bs16_iou03_size896',
 'mrSAdv3_bs16_iou03_size448',
 'mrSAdv3_bs16_iou03_size896',
 'mr_bs16_iou03_size448_base1',
 'mr_bs16_iou03_size448_base2',
 'mr_bs16_iou03_size896',
 'mr_bs16_iou03_siz

In [130]:
key = 'MP_b012_CO_dv0123_bs16_iou03_size896'
if True:
    result_path = '../val_output/N4_All/' + key + '/'
    print(result_path)
    result_list = sorted([each for each in os.listdir(result_path)])

    result_list = sorted([each for each in os.listdir(result_path) if each[0] == 'K'])
    K_pid_score = {}

    for k_idx in range(0,5):
        K_pid_score[str(k_idx)] = {}

        slice_score_list = []
        slice_label_list = []

        tmp_result_npz = [each for each in result_list if each.startswith('K%s'%k_idx)]
        if len(tmp_result_npz) == 0:
            continue
        else:
            tmp_result_npz = tmp_result_npz[0]

    #     print(tmp_result_npz)
        tmp_result_path = result_path + tmp_result_npz
        tmp_result_file = np.load(tmp_result_path, allow_pickle=True)
        case_list = tmp_result_file['case']

        all_detections = tmp_result_file['det']
        all_annotations = tmp_result_file['anno']

        if 'cls' in key:
            model_slice_score_list = tmp_result_file['cls_pre']

        false_positives = np.zeros((0,))
        true_positives = np.zeros((0,))
        scores = np.zeros((0,))
        num_annotations = 0.0

        for i in range(len(case_list)):
            case = case_list[i]
            pid = '_'.join(case.split('_')[:-1])

            if pid not in K_pid_score[str(k_idx)]:
                K_pid_score[str(k_idx)][pid] = [0]

            detections = all_detections[i]
            annotations = all_annotations[i]
            num_annotations += annotations.shape[0]
            detected_annotations = []

            bb_scores = torch.tensor(detections[:,4])
            anchorBoxes = torch.tensor(detections[:,:4])
            anchors_nms_idx = nms(anchorBoxes, bb_scores, 0.1)
            anchors_nms_idx = anchors_nms_idx.numpy()
            detections = detections[anchors_nms_idx]

            for d in detections:
                det_score = d[4]
                K_pid_score[str(k_idx)][pid] += [det_score]

            try:
                slice_score_list.append(np.max(detections[:,4]))
            except:
                slice_score_list.append(0)
            slice_label_list.append(int(annotations.shape[0] != 0))

        for pid, score_list in K_pid_score[str(k_idx)].items():
            K_pid_score[str(k_idx)][pid] = np.mean(sorted(score_list)[-3:])

../val_output/N4_All/MP_b012_CO_dv0123_bs16_iou03_size896/


In [131]:
S_IOU = 0.05
H_IOU = 0.3
NMS_IOU = 0.1
CLS_th = 0.01

result_list = sorted([each for each in os.listdir(result_path) if each[0] == 'K'])
for k_idx in range(0,5):
    tmp_result_npz = [each for each in result_list if each.startswith('K%s'%k_idx)]
    
    if len(tmp_result_npz) == 0:
        continue
    else:
        tmp_result_npz = tmp_result_npz[0]
    tmp_result_path = result_path + tmp_result_npz
    tmp_result_file = np.load(tmp_result_path, allow_pickle=True)
    case_list = tmp_result_file['case']

    all_detections = tmp_result_file['det']
    all_annotations = tmp_result_file['anno']
    
    false_positives = np.zeros((0,))
    true_positives = np.zeros((0,))
    scores = np.zeros((0,))
    num_annotations = 0.0
    
    pos_pid_list = []
    neg_pid_list = []
    pos_score_list = []
    neg_score_list = []

    for i in range(len(case_list)):
        case_name = case_list[i]
        seg_anno_name = '_'.join(case_name.split('_')[:-1])
        
        detections = all_detections[i]
        annotations = all_annotations[i]
        num_annotations += annotations.shape[0]
        detected_annotations = []
  
        if len(detections) == 0:
            if annotations.shape[0] == 0:
                if seg_anno_name[0] == 'I':
                    neg_pid_list.append(seg_anno_name)
                    neg_score_list.append(0)
            continue
        
        boxes_list = [detections[:,:4] / 448]
        scores_list = [detections[:,-1]]
        labels_list = np.ones_like(scores_list)
    
        iou_thr = NMS_IOU
        skip_box_thr = 0.0001
        
        boxes, nms_scores, labels = non_maximum_weighted(boxes_list, 
                                            scores_list, 
                                            labels_list, 
                                            iou_thr=iou_thr,
                                            skip_box_thr=skip_box_thr)
        
        boxes = boxes * 448
        nms_scores = nms_scores[:,np.newaxis]
        detections = np.concatenate([boxes, nms_scores], axis=1)
        
        if len(detections) == 0:
            if annotations.shape[0] == 0:
                if seg_anno_name[0] == 'I':
                    neg_pid_list.append(seg_anno_name)
                    neg_score_list.append(0)
            continue
        
        detections = np.array(sorted(detections.tolist(), key=lambda x:x[-1], reverse=True))

        for d in detections:
            tmp_score = d[4]
            
            if tmp_score < S_IOU:
                continue
            
            tmp_score = tmp_score * K_pid_score[str(k_idx)][seg_anno_name]
            scores = np.append(scores, tmp_score)
            
            if annotations.shape[0] == 0:
                false_positives = np.append(false_positives, 1)
                true_positives = np.append(true_positives, 0)
                
                if seg_anno_name[0] == 'I':
                    neg_pid_list.append(seg_anno_name)
                    neg_score_list.append(tmp_score)
                continue

            d_tensor = torch.tensor(d[:4][np.newaxis])
            a_tensor = torch.tensor(annotations)
            overlaps = box_iou(d_tensor, a_tensor).numpy()
            assigned_annotation = np.argmax(overlaps, axis=1)
            max_overlap = overlaps[0, assigned_annotation]

            if max_overlap >= H_IOU:
                if assigned_annotation not in detected_annotations:
                    false_positives = np.append(false_positives, 0)
                    true_positives = np.append(true_positives, 1)
                    detected_annotations.append(assigned_annotation)
                    
                    if seg_anno_name[0] != 'I':
                        pos_pid_list.append(seg_anno_name)
                        pos_score_list.append(tmp_score)
                else:
                    false_positives = np.append(false_positives, 1)
                    true_positives = np.append(true_positives, 0)
            else:
                false_positives = np.append(false_positives, 1)
                true_positives = np.append(true_positives, 0)
                if seg_anno_name[0] == 'I':
                    neg_pid_list.append(seg_anno_name)
                    neg_score_list.append(tmp_score)
    
    anno_list[k_idx] = num_annotations
    if len(false_positives) == 0 and len(true_positives) == 0:
        print('No detection')
    else:
        # sort by score
        indices = np.argsort(-scores)
        scores = scores[indices]
        false_positives = false_positives[indices]
        true_positives = true_positives[indices]
        
        anno = num_annotations
        case_num = len(case_list)

        # compute false positives and true positives
        false_positives = np.cumsum(false_positives)
        true_positives = np.cumsum(true_positives)

        # compute recall and precision
        recall = true_positives / num_annotations
        precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)
        
        # compute average precision
        average_precision = compute_ap(recall, precision)
        
        recall_copy = deepcopy(recall)
        recall_copy[recall_copy < 0.8] = 0
        recall_th_idx = np.argmin(np.abs(recall_copy - 0.8))
        
        recall_max = recall[recall_th_idx]
        precision_max = precision[recall_th_idx]
        while precision[recall_th_idx] >= precision_max:
            recall_th_idx += 1
            
        recall = recall[recall_th_idx]
        precision = precision[recall_th_idx]

        print('mAP: {:.4f}'.format(average_precision))
        print("Precision: {:.4f}".format(precision))
        print("Recall: {:.4f}".format(recall))
        print("F1-Score: {:.4f}".format(2*recall*precision/(recall+precision)))

        fp_list = false_positives
        tp_list = true_positives
        
        fps_list = []
        
        for th in [0.05, 0.125, 0.25, 0.5, 1]:
            fp_th_idx = np.argmin(np.abs(fp_list / case_num - th))
            tp_th = tp_list[fp_th_idx]
            print('%s - %1.3f'%(scores[fp_th_idx], th), (tp_th / anno).round(4))
            fps_list.append(tp_th / anno)
        print('Mean:', np.mean(fps_list).round(4))
    
    unique_score_list = sorted(list(set(pos_score_list + neg_score_list)))
    unique_score_list.insert(0, -1)
    unique_score_list.append(1.1)
    
    unique_pid_list = np.unique(np.array(neg_pid_list).tolist())
    Neg_pid_num = len(np.unique(np.array(neg_pid_list).tolist()))

    sens_case_list = []
    spec_pid_list = []

    for th_score in unique_score_list:
        TP_case_num = np.sum(np.array(pos_score_list) > th_score)
        sens_case = TP_case_num / num_annotations
        sens_case_list.append(sens_case)
        
        FP_pid_num = len(np.unique(np.array(neg_pid_list)[np.array(neg_score_list) > th_score]).tolist())
        spec_pid_list.append(FP_pid_num/Neg_pid_num)

#     for sens_idx, sens in enumerate(sens_case_list):
#         print(sens, spec_pid_list[sens_idx])
    
    
#     fig = plt.figure(figsize=(10,10),dpi=100)
#     plt.plot(spec_pid_list, sens_case_list)
#     plt.xlim(-0.1,1.1)
#     plt.ylim(-0.1,1.1)
#     plt.show()
#     break
    
    print('AFROC:', auc(spec_pid_list, sens_case_list).round(4))
    print('========================================')


mAP: 0.8154
Precision: 0.7345
Recall: 0.8023
F1-Score: 0.7669
0.691773969779336 - 0.050 0.6145
0.5122357997414468 - 0.125 0.7661
0.3378085749131602 - 0.250 0.8715
0.14588923199588893 - 0.500 0.9259
0.026256117531664902 - 1.000 0.9572
Mean: 0.827
AFROC: 0.7671
mAP: 0.7746
Precision: 0.6645
Recall: 0.8010
F1-Score: 0.7264
0.8596325400043069 - 0.050 0.4777
0.7792338336879467 - 0.125 0.6959
0.6443295365123495 - 0.250 0.8328
0.41234382731968344 - 0.500 0.9156
0.14441968744893546 - 1.000 0.9554
Mean: 0.7755
AFROC: 0.7933
mAP: 0.8367
Precision: 0.7973
Recall: 0.8009
F1-Score: 0.7991
0.8749577241277748 - 0.050 0.6462
0.7670428954002659 - 0.125 0.8147
0.5586572803985371 - 0.250 0.8729
0.26805680454222625 - 0.500 0.9127
0.05040768923748843 - 1.000 0.9342
Mean: 0.8361
AFROC: 0.856
mAP: 0.7810
Precision: 0.7043
Recall: 0.8007
F1-Score: 0.7494
0.9125222576362961 - 0.050 0.4679
0.8028733250334632 - 0.125 0.7249
0.5136689174879147 - 0.250 0.8567
0.1543423164586244 - 0.500 0.9176
0.006176784141477853 

In [132]:
S_IOU = 0.05
H_IOU = 0.3
NMS_IOU = 0.1
CLS_th = 0.01

false_positives = np.zeros((0,))
true_positives = np.zeros((0,))
scores = np.zeros((0,))
num_annotations = 0.0
case_num = 0

pos_pid_list = []
neg_pid_list = []
pos_score_list = []
neg_score_list = []

if True:
    result_list = sorted([each for each in os.listdir(result_path) if each[0] == 'K'])
    for k_idx in range(0,5):
        tmp_result_npz = [each for each in result_list if each.startswith('K%s'%k_idx)]

        if len(tmp_result_npz) == 0:
            continue
        else:
            tmp_result_npz = tmp_result_npz[0]
        tmp_result_path = result_path + tmp_result_npz
        tmp_result_file = np.load(tmp_result_path, allow_pickle=True)
        case_list = tmp_result_file['case']
        case_num += len(case_list)

        all_detections = tmp_result_file['det']
        all_annotations = tmp_result_file['anno']

        for i in range(len(case_list)):
            case_name = case_list[i]
            seg_anno_name = '_'.join(case_name.split('_')[:-1])

            detections = all_detections[i]
            annotations = all_annotations[i]
            num_annotations += annotations.shape[0]
            detected_annotations = []

            if len(detections) == 0:
                if annotations.shape[0] == 0:
                    if seg_anno_name[0] == 'I':
                        neg_pid_list.append(seg_anno_name)
                        neg_score_list.append(0)
                continue

            boxes_list = [detections[:,:4] / 448]
            scores_list = [detections[:,-1]]
            labels_list = np.ones_like(scores_list)

            iou_thr = NMS_IOU
            skip_box_thr = 0.0001

            boxes, nms_scores, labels = non_maximum_weighted(boxes_list, 
                                                scores_list, 
                                                labels_list, 
                                                iou_thr=iou_thr,
                                                skip_box_thr=skip_box_thr)

            boxes = boxes * 448
            nms_scores = nms_scores[:,np.newaxis]
            detections = np.concatenate([boxes, nms_scores], axis=1)

            if len(detections) == 0:
                if annotations.shape[0] == 0:
                    if seg_anno_name[0] == 'I':
                        neg_pid_list.append(seg_anno_name)
                        neg_score_list.append(0)
                continue

            detections = np.array(sorted(detections.tolist(), key=lambda x:x[-1], reverse=True))

            for d in detections:
                tmp_score = d[4]

                if tmp_score < S_IOU:
                    continue

                tmp_score = tmp_score * K_pid_score[str(k_idx)][seg_anno_name]
                scores = np.append(scores, tmp_score)

                if annotations.shape[0] == 0:
                    false_positives = np.append(false_positives, 1)
                    true_positives = np.append(true_positives, 0)

                    if seg_anno_name[0] == 'I':
                        neg_pid_list.append(seg_anno_name)
                        neg_score_list.append(tmp_score)
                    continue

                d_tensor = torch.tensor(d[:4][np.newaxis])
                a_tensor = torch.tensor(annotations)
                overlaps = box_iou(d_tensor, a_tensor).numpy()
                assigned_annotation = np.argmax(overlaps, axis=1)
                max_overlap = overlaps[0, assigned_annotation]

                if max_overlap >= H_IOU:
                    if assigned_annotation not in detected_annotations:
                        false_positives = np.append(false_positives, 0)
                        true_positives = np.append(true_positives, 1)
                        detected_annotations.append(assigned_annotation)

                        if seg_anno_name[0] != 'I':
                            pos_pid_list.append(seg_anno_name)
                            pos_score_list.append(tmp_score)
                    else:
                        false_positives = np.append(false_positives, 1)
                        true_positives = np.append(true_positives, 0)
                else:
                    false_positives = np.append(false_positives, 1)
                    true_positives = np.append(true_positives, 0)
                    if seg_anno_name[0] == 'I':
                        neg_pid_list.append(seg_anno_name)
                        neg_score_list.append(tmp_score)

    if len(false_positives) == 0 and len(true_positives) == 0:
        print('No detection')
    else:
        # sort by score
        indices = np.argsort(-scores)
        scores = scores[indices]
        false_positives = false_positives[indices]
        true_positives = true_positives[indices]

        anno = num_annotations

        # compute false positives and true positives
        false_positives = np.cumsum(false_positives)
        true_positives = np.cumsum(true_positives)

        # compute recall and precision
        recall = true_positives / num_annotations
        precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)

        # compute average precision
        average_precision = compute_ap(recall, precision)

        recall_copy = deepcopy(recall)
        recall_copy[recall_copy < 0.8] = 0
        recall_th_idx = np.argmin(np.abs(recall_copy - 0.8))

        recall_max = recall[recall_th_idx]
        precision_max = precision[recall_th_idx]
        while precision[recall_th_idx] >= precision_max:
            recall_th_idx += 1

        recall = recall[recall_th_idx]
        precision = precision[recall_th_idx]

        print('mAP: {:.4f}'.format(average_precision))
        print("Precision: {:.4f}".format(precision))
        print("Recall: {:.4f}".format(recall))
        print("F1-Score: {:.4f}".format(2*recall*precision/(recall+precision)))

        fp_list = false_positives
        tp_list = true_positives

        fps_list = []

        for th in [0.05, 0.125, 0.25, 0.5, 1]:
            fp_th_idx = np.argmin(np.abs(fp_list / case_num - th))
            tp_th = tp_list[fp_th_idx]
            print('%s - %1.3f'%(scores[fp_th_idx], th), (tp_th / anno).round(4))
            fps_list.append(tp_th / anno)
        print('Mean:', np.mean(fps_list).round(4))

    unique_score_list = sorted(list(set(pos_score_list + neg_score_list)))
    unique_score_list.insert(0, -1)
    unique_score_list.append(1.1)

    unique_pid_list = np.unique(np.array(neg_pid_list).tolist())
    Neg_pid_num = len(np.unique(np.array(neg_pid_list).tolist()))

    sens_case_list = []
    spec_pid_list = []

    for th_score in unique_score_list:
        TP_case_num = np.sum(np.array(pos_score_list) > th_score)
        sens_case = TP_case_num / num_annotations
        sens_case_list.append(sens_case)

        FP_pid_num = len(np.unique(np.array(neg_pid_list)[np.array(neg_score_list) > th_score]).tolist())
        spec_pid_list.append(FP_pid_num/Neg_pid_num)

#     fig = plt.figure(figsize=(10,10),dpi=100)
#     plt.plot(spec_pid_list, sens_case_list)
#     plt.xlim(-0.1,1.1)
#     plt.ylim(-0.1,1.1)
#     plt.show()

    print('AFROC:', auc(spec_pid_list, sens_case_list).round(4))
    print('========================================')

mAP: 0.7793
Precision: 0.6984
Recall: 0.8000
F1-Score: 0.7457
0.8723435002650959 - 0.050 0.5116
0.7494993621327122 - 0.125 0.7327
0.5166639257073864 - 0.250 0.8551
0.2412618833966723 - 0.500 0.9149
0.058683584976737535 - 1.000 0.9452
Mean: 0.7919
AFROC: 0.8341


In [111]:
H_IOU = 0.3
NMS_IOU = 0.01
top = 3
th = 0.7

result_list = sorted([each for each in os.listdir(result_path) if each[0] == 'K'])
for k_idx in range(0,5):
    tmp_result_npz = [each for each in result_list if each.startswith('K%s'%k_idx)]
    if len(tmp_result_npz) == 0:
        continue
    else:
        tmp_result_npz = tmp_result_npz[0]
    tmp_result_path = result_path + tmp_result_npz
    tmp_result_file = np.load(tmp_result_path, allow_pickle=True)
    case_list = tmp_result_file['case']

    all_detections = tmp_result_file['det']
    all_annotations = tmp_result_file['anno']
    
    false_positives = np.zeros((0,))
    true_positives = np.zeros((0,))
    scores = np.zeros((0,))
    num_annotations = 0.0
    
    pid_dict = {}

    for i in range(len(case_list)):
        case = case_list[i]
        pid = '_'.join(case.split('_')[:-1])
        
        if pid not in pid_dict:
            pid_dict[pid] = [0]
        
        detections = all_detections[i]
        annotations = all_annotations[i]
        num_annotations += annotations.shape[0]
        detected_annotations = []
        
        if len(detections) == 0:
            if annotations.shape[0] == 0:
                if seg_anno_name[0] == 'I':
                    neg_pid_list.append(seg_anno_name)
                    neg_score_list.append(0)
            continue
        
#         bb_scores = torch.tensor(detections[:,4])
#         anchorBoxes = torch.tensor(detections[:,:4])
#         anchors_nms_idx = nms(anchorBoxes, bb_scores, NMS_IOU)
#         anchors_nms_idx = anchors_nms_idx.numpy()
#         detections = detections[anchors_nms_idx]
        
        boxes_list = [detections[:,:4] / 448]
        scores_list = [detections[:,-1]]
        labels_list = np.ones_like(scores_list)
    
        iou_thr = NMS_IOU
        skip_box_thr = 0.0001
        
        boxes, nms_scores, labels = non_maximum_weighted(boxes_list, 
                                            scores_list, 
                                            labels_list, 
                                            iou_thr=iou_thr,
                                            skip_box_thr=skip_box_thr)
        
        boxes = boxes * 448
        nms_scores = nms_scores[:,np.newaxis]
        detections = np.concatenate([boxes, nms_scores], axis=1)
        
        for d in detections:
            det_score = d[4] * K_pid_score[str(k_idx)][pid]
            pid_dict[pid] += [det_score]

    score_list = []
    label_list = []
    for pid in pid_dict:
        if pid[:2] == 'I_':
            label_list.append(0)
        else:
            label_list.append(1)
        score_list.append(np.mean(sorted(pid_dict[pid], reverse=True)[:top]))
        
    score_list = np.array(score_list)
    label_list = np.array(label_list)
    
    print(round(roc_auc_score(label_list, score_list),4))
    print(round(accuracy_score(label_list, score_list>th),4))
    print(round(recall_score(label_list, score_list>th),4))
    print(round(recall_score(1-label_list, score_list<=th),4))
    print('======================================')

0.9551
0.9492
0.9592
0.9
0.9755
0.8983
0.9592
0.6
0.9612
0.8644
0.898
0.7
0.9773
0.7586
0.7143
1.0
0.9583
0.8421
0.8333
0.8889


In [ ]:
for H_IOU in [0.1, 0.2, 0.3, 0.4, 0.5]:
    print('=========== Hit IoU %s ========='%H_IOU)
    S_IOU = 0.05
    NMS_IOU = 0.01

    result_list = sorted([each for each in os.listdir(result_path) if each[0] == 'K'])
    for k_idx in range(0,5):
        tmp_result_npz = [each for each in result_list if each.startswith('K%s'%k_idx)]
        if len(tmp_result_npz) == 0:
            continue
        else:
            tmp_result_npz = tmp_result_npz[0]
        tmp_result_path = result_path + tmp_result_npz
        tmp_result_file = np.load(tmp_result_path, allow_pickle=True)
        case_list = tmp_result_file['case']

        all_detections = tmp_result_file['det']
        all_annotations = tmp_result_file['anno']

        false_positives = np.zeros((0,))
        true_positives = np.zeros((0,))
        scores = np.zeros((0,))
        num_annotations = 0.0

        for i in range(len(case_list)):
            case = case_list[i]
            pid = '_'.join(case.split('_')[:-1])
            
            detections = all_detections[i]
            annotations = all_annotations[i]
            num_annotations += annotations.shape[0]
            detected_annotations = []

            boxes_list = [detections[:,:4] / 448]
            scores_list = [detections[:,-1]]
            labels_list = np.ones_like(scores_list)

            iou_thr = NMS_IOU
            skip_box_thr = 0.0001

            boxes, nms_scores, labels = non_maximum_weighted(boxes_list, 
                                                scores_list, 
                                                labels_list, 
                                                iou_thr=iou_thr,
                                                skip_box_thr=skip_box_thr)

            boxes = boxes * 448
            nms_scores = nms_scores[:,np.newaxis]
            detections = np.concatenate([boxes, nms_scores], axis=1)

            for d in detections:
                if d[4] < S_IOU:
                    continue
                
                tmp_score = d[4]  * K_pid_score[str(k_idx)][pid]
                scores = np.append(scores, tmp_score)

                if annotations.shape[0] == 0:
                    false_positives = np.append(false_positives, 1)
                    true_positives = np.append(true_positives, 0)
                    continue

                d_tensor = torch.tensor(d[:4][np.newaxis])
                a_tensor = torch.tensor(annotations)
                overlaps = box_iou(d_tensor, a_tensor).numpy()
                assigned_annotation = np.argmax(overlaps, axis=1)
                max_overlap = overlaps[0, assigned_annotation]

                if max_overlap >= H_IOU and assigned_annotation not in detected_annotations:
                    false_positives = np.append(false_positives, 0)
                    true_positives = np.append(true_positives, 1)
                    detected_annotations.append(assigned_annotation)
                else:
                    false_positives = np.append(false_positives, 1)
                    true_positives = np.append(true_positives, 0)

        if len(false_positives) == 0 and len(true_positives) == 0:
            print('No detection')
        else:
            # sort by score
            indices = np.argsort(-scores)
            scores = scores[indices]
            false_positives = false_positives[indices]
            true_positives = true_positives[indices]

            anno = num_annotations
            case_num = len(case_list)

            # compute false positives and true positives
            false_positives = np.cumsum(false_positives)
            true_positives = np.cumsum(true_positives)

            # compute recall and precision
            recall = true_positives / num_annotations
            precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)

            # compute average precision
            average_precision = compute_ap(recall, precision)

            print('mAP: {:.4f}'.format(average_precision))